In [ ]:
import h5py

with h5py.File(r'D:\Projects\SARS-Prediction_v2\preprocess\ESM\output\sampled_output_esm_embeddings.h5', 'r') as f:
    root = f
    print(root.keys())  # 打印所有的键
    print(root['indices'].shape)  # 前10个样本索引
#     # print(root['sequences'][:10])  # 前10个序列
#     # 访问第一个batch的第一个embedding
    # emb = root['attrs']
    # print(emb)  # 打印所有的键

<KeysViewHDF5 ['diff_counts', 'embeddings', 'indices', 'mean_log10Ka', 'seq_lengths', 'sequences']>
(50,)


In [7]:
with h5py.File(r'D:\Projects\SARS-Prediction_v2\preprocess\ESM\output\sampled_output_esm_embeddings.h5', 'r') as f:
    def print_structure(name, obj):
        print(name)
    f.visititems(print_structure)

diff_count
embeddings
embeddings/emb_0
embeddings/emb_1
embeddings/emb_10
embeddings/emb_100
embeddings/emb_1000
embeddings/emb_10000
embeddings/emb_10001
embeddings/emb_10002
embeddings/emb_10003
embeddings/emb_10004
embeddings/emb_10005
embeddings/emb_10006
embeddings/emb_10007
embeddings/emb_10008
embeddings/emb_10009
embeddings/emb_1001
embeddings/emb_10010
embeddings/emb_10011
embeddings/emb_10012
embeddings/emb_10013
embeddings/emb_10014
embeddings/emb_10015
embeddings/emb_10016
embeddings/emb_10017
embeddings/emb_10018
embeddings/emb_10019
embeddings/emb_1002
embeddings/emb_10020
embeddings/emb_10021
embeddings/emb_10022
embeddings/emb_10023
embeddings/emb_10024
embeddings/emb_10025
embeddings/emb_10026
embeddings/emb_10027
embeddings/emb_10028
embeddings/emb_10029
embeddings/emb_1003
embeddings/emb_10030
embeddings/emb_10031
embeddings/emb_10032
embeddings/emb_10033
embeddings/emb_10034
embeddings/emb_10035
embeddings/emb_10036
embeddings/emb_10037
embeddings/emb_10038
embeddin

In [6]:
import h5py

with h5py.File(r'D:\Projects\SARS-Prediction_v2\preprocess\ESM\output\sampled_output_esm_embeddings_debug.h5', 'r') as f:
    indices = f['indices'][:]
    embeddings_group = f['embeddings']
    missing = []
    for idx in indices:
        emb_name = f'emb_{idx}'
        if emb_name not in embeddings_group:
            missing.append(emb_name)
    if missing:
        print(f"缺失的embedding数据集: {missing}")
    else:
        print("所有indices都有对应的emb_{index}数据集。")

所有indices都有对应的emb_{index}数据集。


In [ ]:
import h5py
import numpy as np
import concurrent.futures
import os

input_path = r"D:\Projects\SARS-Prediction_v2\preprocess/ESM/output/sampled_output_esm_embeddings.h5"
output_path = r"D:\Projects\SARS-Prediction_v2\preprocess/ESM/output/sampled_output_esm_embeddings_merged.h5"
BATCH_WRITE_SIZE = 500
MAX_WORKERS = 8

# 首先检查并打印文件结构，以了解实际的层次结构
def print_hdf5_structure(name, obj):
    print(f"找到: {name} ({'组' if isinstance(obj, h5py.Group) else '数据集'})")

print("HDF5文件结构:")
with h5py.File(input_path, 'r') as f:
    f.visititems(print_hdf5_structure)
    
# 然后根据文件实际结构修改代码
def copy_embedding(args):
    emb_path, emb_name = args
    try:
        with h5py.File(input_path, 'r') as fin:
            emb = fin[emb_path][:]
            with h5py.File(output_path, 'a') as fout:
                embeddings_out = fout['protein_embeddings/embeddings']
                if emb_name not in embeddings_out:
                    embeddings_out.create_dataset(emb_name, data=emb)
    except Exception as e:
        print(f"处理 {emb_path} 时出错: {e}")

# 创建新文件
with h5py.File(output_path, 'w') as fout:
    root_out = fout.create_group('protein_embeddings')
    embeddings_out = root_out.create_group('embeddings')

    # 先从输入文件复制全局元数据
    with h5py.File(input_path, 'r') as fin:
        root_in = fin['protein_embeddings']
        
        # 复制全局数据集
        for key in ['indices', 'diff_counts', 'mean_log10Ka', 'seq_lengths', 'sequences']:
            if key in root_in:
                data = root_in[key][:]
                root_out.create_dataset(key, data=data)
        
        # 复制属性
        for attr in root_in.attrs:
            root_out.attrs[attr] = root_in.attrs[attr]
        
        # 收集所有embedding路径
        emb_tasks = []
        global_idx = 0
        
        # 根据第一步检查的结构修改此处代码
        print("收集嵌入向量路径...")
        for key in root_in:
            if key.startswith('batch_'):
                # 检查batch下是否有embeddings子组
                batch = root_in[key]
                if 'embeddings' in batch:
                    emb_group = batch['embeddings']
                    for emb_key in emb_group:
                        emb_path = f'protein_embeddings/{key}/embeddings/{emb_key}'
                        emb_name = f'emb_{global_idx}'
                        emb_tasks.append((emb_path, emb_name))
                        global_idx += 1
                else:
                    # 直接检查batch是否包含数据集
                    for emb_key in batch:
                        if isinstance(batch[emb_key], h5py.Dataset):
                            emb_path = f'protein_embeddings/{key}/{emb_key}'
                            emb_name = f'emb_{global_idx}'
                            emb_tasks.append((emb_path, emb_name))
                            global_idx += 1
        
        print(f"找到 {len(emb_tasks)} 个嵌入向量需要处理")

# 分批多线程处理
for i in range(0, len(emb_tasks), BATCH_WRITE_SIZE):
    batch = emb_tasks[i:i+BATCH_WRITE_SIZE]
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        list(executor.map(copy_embedding, batch))
    print(f"已处理 {min(i+BATCH_WRITE_SIZE, len(emb_tasks))}/{len(emb_tasks)} 个embeddings")

# 验证结果
with h5py.File(output_path, 'r') as f:
    count = len(f['protein_embeddings/embeddings'])
    print(f"合并完成，共整合了 {count} 个embedding到 {output_path} 的 protein_embeddings/embeddings 下。")
    print(f"文件大小: {os.path.getsize(output_path)/1024/1024:.2f} MB")


In [ ]:
import h5py

input_path = r"D:\Projects\SARS-Prediction_v2\preprocess/ESM/output/sampled_output_esm_embeddings.h5"

